In [19]:
%reload_ext autoreload
%autoreload 2
import torch
from torch import nn
from models import MyLSTM
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tqdm import tqdm

In [20]:
model = MyLSTM(hidden_size=100)

In [21]:
from pendulum import DoublePendulum

In [22]:
pendulum = DoublePendulum()

The double pendulum is initialized with the following physical parameters:
m1 =  2 kg m2 =  1 kg l1 =  1.4 m l2 =  1 m g =  9.8 m/s^2 number of points =  1500


In [23]:
trajectory = pendulum.compute_trajectory(theta1=-60, theta2=60, number_of_points=1500, simulation_time=25.0)
number_of_points = trajectory.shape[0]

Trajectory computed


In [24]:
number_of_points

1500

In [25]:
trajectory.shape

(1500, 4)

In [26]:
#the functions below are used to create the training set
lookback = 10

def create_inout_sequences(input_data, lookback):
    inout_seq = []
    L = len(input_data)
    for i in range(L-lookback):
        train_seq = input_data[i:i+lookback]
        train_label = input_data[i+lookback:i+lookback+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

def preprocess_trajectory(trajectory, train_window):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    train_data_normalized = scaler.fit_transform(trajectory.reshape(-1, 4))
    train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)
    train_inout_seq= create_inout_sequences(train_data_normalized.reshape(number_of_points, 4), lookback)
    return train_inout_seq

train_inout_seq = preprocess_trajectory(trajectory, lookback)
len(train_inout_seq)

1490

In [27]:
loss_function = nn.MSELoss()
hidden_layer_size = 100
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
epochs =2000

In [28]:
for i in range(epochs):
    seq_err = 0.0
    progress_bar = tqdm(range(500), desc=f"Epoch {i}")
    for _ in progress_bar:
        num = np.random.randint(len(train_inout_seq))
        seq = train_inout_seq[num][0]
        labels = train_inout_seq[num][1]
        optimizer.zero_grad()
        model.c_h = (torch.zeros(1, 1, hidden_layer_size),
                        torch.zeros(1, 1, hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred.reshape(1,4), labels)
        single_loss.backward()
        optimizer.step()
        seq_err+= single_loss.item()
        
        progress_bar.set_postfix({"loss": seq_err})

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')
        print(f'seq: {i:3} loss: {seq_err:10.8f}')
    if seq_err < 0.000007:
        break

Epoch 1: 100%|██████████| 500/500 [00:01<00:00, 370.55it/s, loss=4.13]


epoch:   1 loss: 0.00827176
seq:   1 loss: 4.13435815


Epoch 26: 100%|██████████| 500/500 [00:01<00:00, 350.24it/s, loss=0.146] 


epoch:  26 loss: 0.00011526
seq:  26 loss: 0.14609162


Epoch 51: 100%|██████████| 500/500 [00:01<00:00, 377.56it/s, loss=0.0744]


...

epoch: 1051 loss: 0.00000649
seq: 1051 loss: 0.00204566


Epoch 1076: 100%|██████████| 500/500 [00:01<00:00, 339.56it/s, loss=0.00492] 


epoch: 1076 loss: 0.00000130
seq: 1076 loss: 0.00491523


Epoch 1096:  66%|██████▌   | 328/500 [00:00<00:00, 330.83it/s, loss=0.00227] 
[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument


KeyboardInterrupt: 

In [29]:
print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch: 1096 loss: 0.0000018193


In [30]:
torch.save(model.state_dict(), 'models/lstm')